In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

In [23]:
from openai import OpenAI
import random

def get_response(msg):
    messages = [
        {
            "role": "user",
            "content": msg
        }
    ]
    
    temperature = random.uniform(0.7, 1.0)
    client = OpenAI(
        api_key=os.environ["OPENAI_API_KEY"],
        base_url=os.environ["OPENAI_API_BASE"],
    )
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=messages,
        temperature=temperature
    )
    
    output = response.choices[0].message.content
    return output

In [24]:
get_response("What is the meaning of life?")

'The question "What is the meaning of life?" is one of the most profound and enduring questions in human philosophy and religion. Its answer varies depending on cultural, individual, philosophical, and religious beliefs. Here are some of the perspectives people might offer:\n\n1. Religious Perspectives: Many religions offer their own answers to the meaning of life. For example, in many strands of Christianity, the meaning of life is to follow the will and teachings of God, leading to eternal salvation. In Buddhism, it may be to achieve enlightenment and end the cycle of suffering and rebirth.\n\n2. Philosophical Perspectives: Philosophers have proposed various interpretations. Existentialists like Jean-Paul Sartre suggest that life inherently has no meaning, and it is up to individuals to create their own meaning. In contrast, philosophers such as Aristotle believed that the purpose of life is to achieve eudaimonia, often translated as happiness or human flourishing.\n\n3. Scientific P

In [25]:
import pandas as pd
df = pd.read_csv('../kaggle_dataset/archive/lb_related_5k.csv')
df = df[:3]

In [6]:
df.head()

,rewrite_prompt,original_text,rewritten_text
0,Use more descriptive language to enhance the a...,The relationship between sustainability and to...,"Sure, here is a more descriptive version of th..."
1,Use more descriptive language to enhance the a...,This study tested the effect of voter's politi...,"Sure, here is a version of the text with more ..."
2,Use more descriptive language to enhance the a...,While onsite measurement of air permeability p...,"Sure, here is the text with more descriptive l..."


In [7]:
# def g(df, col):
#     return df.groupby(col).apply(lambda x: x.sample(1)).reset_index(drop=True)
# result = g(df.copy(), 'rewrite_prompt')
result = df.copy()

In [8]:
def formatting_prompts_func(examples):
    inst = """
You'll be given an original text and a rewritten text generated by an LLM. The prompt that guided the LLM's changes will be given as well. 
Suppose you don't know the original prompt, your task is to analyze the differences between an original text and a rewritten version generated by an LLM and try to infer it based on the differences you see. 
Provide a detailed explanation of how you arrived at your inference step by step.

**Original Text**:
{}

**Prompt**:
{}

**Rewritten Text**
{}

You should response in the following format:
**Inferred Promp**: ...

**Chain of Thoughts**: ...
"""
    return inst.format(examples['original_text'], examples['rewrite_prompt'], examples['rewritten_text'])

In [9]:
cot = []
import time
from tqdm import tqdm
for idx, row in tqdm(result.iterrows(), total = len(result)):
    #输入提示
    try:
        prompt_parts = formatting_prompts_func(row)
        #输出回答
        while True:
            try:
                response = get_response(prompt_parts)
                break
            except Exception as e:
                print(e)
                time.sleep(1)

        cot.append(response)
#         print(response)
    except:
        cot.append("")
    
result['cot'] = cot 
result.to_csv('./datasets_with_cot.csv', index = False)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [02:19<00:00, 46.62s/it]


In [11]:
result['cot'].values

array(['**Inferred Prompt**: Enhance the text with metaphorical and evocative language to better convey the complex interplay between sustainability and tourism.\n\n**Chain of Thoughts**: \n- The rewritten text employs more vibrant and metaphorical language such as "a complex dance," "woven through intricate threads," "daunting task," "pinpoint the factors that nudge," "shelling out extra dough," "illuminate this complex landscape," "a ray of light," "decipher," "orchestrate," "canvas unfolds," and "grappling with the complexities." These phrases are not present in the original text and serve to create a more vivid and descriptive atmosphere.\n- The use of "a complex dance" and "woven through intricate threads" suggests movement and an interwoven nature, which is a significant embellishment from the simple "complex" in the original text.\n- "Stagnant" replaces "little increase," further personifying the state of research studies as motionless water, which adds to the atmospheric qualit

In [6]:
import aiohttp
import json
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

# os.environ["OPENAI_API_BASE"] = 'http://gpt-proxy.jd.com/gateway/azure/chat/completions'
# os.environ["OPENAI_API_KEY"] = "6730831d-ee43-4505-98ba-71ed0115ef68"

async def get_response_async(msg, model="gpt-4-1106-preview"):
    url = 'http://gpt-proxy.jd.com/gateway/azure/chat/completions'
    payload = json.dumps({
        "messages": [
            {
                "role": "user",
                "content": msg
            }
        ],
        "stream": False,
        "model": model,
        "temperature": 0.5,
        "presence_penalty": 0,
        "frequency_penalty": 0,
        "max_tokens": 2000
    })
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + os.environ["OPENAI_API_KEY"]
    }

    async with aiohttp.ClientSession() as session:
        for _ in range(5):
            async with session.post(url, headers=headers, data=payload) as response:
                if response.status == 200:
                    text = await response.text()
                    res = json.loads(text)
                    try:
                        return res['choices'][0]['message']['content']
                    except KeyError:
                        if _ != 4:
                            print(f"Request failed, retrying... {res}")
                            continue
                        else:
                            return res
                else:
                    print(
                        f"Request failed with status {response.status}, retrying...")
    return None

In [7]:
import asyncio

async def test_get_response_async():
    msg = "你好吗？"
    response = await get_response_async(msg)
    print("Response:", response)


await test_get_response_async()

Response: 你好！我是一个人工智能助理，所以我没有感觉，但我很乐意帮助你。有什么我可以帮你的吗？


In [47]:
import json
import requests
#import ujson as json
import openai
import os

os.environ["OPENAI_API_BASE"] = 'http://gpt-proxy.jd.com/gateway/azure/chat/completions'

def query_gpt():
    # http方式调用必须使用完整的url（区别于使用openai的sdk调用方式）
    api_key = os.environ["OPENAI_API_KEY"]
    server = os.environ["OPENAI_API_BASE"]
    data = {
        "model": "gpt-3.5-turbo-1106",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "hello"
            }
        ],
        "temperature": 0.5
    }
    json_data = json.dumps(data)
    header = {
        "Content-Type": "application/json;charset=UTF-8",
        "Authorization": f"Bearer {api_key}"
    }
    try:
        resp = requests.post(url=server, data=json_data, headers=header)
        if resp.status_code == 200:
            ret = resp.json()
            return ret
    except Exception as err:
        print(err)
        return []


if __name__ == '__main__':
    print(query_gpt())

{'created': 1710440284, 'usage': {'completion_tokens': 9, 'prompt_tokens': 18, 'total_tokens': 27}, 'model': 'gpt-35-turbo', 'id': 'chatcmpl-92jwSG5IfPxbkebIWs5IDIPiR7Xf6', 'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}}], 'system_fingerprint': 'fp_68a7d165bf', 'object': 'chat.completion'}
